In [1]:
import os
import pandas as pd

from src.squad_selection.select_team import select_team
from src.squad_selection.select_team import print_selected_team

In [2]:
# load data/raw/Fantasy-Premier-League/2022-23/cleaned_players.csv
root_dir = os.path.dirname(os.path.abspath(''))
df_season_start = pd.read_csv(root_dir + '/data/raw/Fantasy-Premier-League/2022-23/players_raw.csv')
teams = pd.read_csv(root_dir + '/data/raw/Fantasy-Premier-League/2022-23/teams.csv')

In [3]:
df_season_start['name'] = df_season_start['first_name'] + ' ' + df_season_start['second_name']

In [4]:
df_season_start.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,cost_change_event,...,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards,name
0,0,0,0,None,None,0,58822,5,NaN,0,...,0,3518,296,18466,1139,0.0,0.0,Cédric,0,Cédric Alves Soares
1,2,2,95,None,None,2,84450,None,NaN,0,...,24,303352,30131,134055,25786,1.0,4.8,Xhaka,1,Granit Xhaka
2,0,0,15,0,25,0,153256,None,NaN,0,...,2,90062,710,126457,18264,0.1,0.5,Elneny,0,Mohamed Elneny
3,0,0,8,None,None,0,156074,None,NaN,0,...,2,2480,342,10371,638,0.1,0.5,Holding,0,Rob Holding
4,0,0,39,0,25,2,167199,None,NaN,0,...,8,25368,103,57694,6745,0.3,1.6,Partey,0,Thomas Partey


## Conclusion
There is problem with selecting starting squad for new season, since now we have only premier league data, so there is lask of stats for new players in the league. I'll try to use data about percent of ownership and value to create some simple predictions based on these values.

In [381]:
df_start = df_season_start.copy()

In [382]:
df_start = df_start[['name', 'id', 'now_cost', 'team', 'element_type', 'selected_by_percent']]

In [383]:
df_start.rename(columns={'id': 'element','now_cost': 'value', 'element_type': 'position_id'}, inplace=True)

In [384]:
# replace df_start['team'] with teams['name'] where teams['id'] == df_start['team']
df_start['team'] = df_start.apply(lambda row: teams.loc[teams['id'] == row['team'], 'name'].values[0], axis=1)
df_start['value'] = df_start['value'] / 10
df_start['position'] = df_start['position_id'].map({1: 'GK', 2: 'DEF', 3: 'MID', 4: 'FWD'})

Create predicted points by multiplying value and ownership percent.

In [385]:
df_start['predicted_total_points_next_gameweek'] = df_start['value'] * df_start['selected_by_percent'] / 10

In [386]:
df_start.sort_values(by=['selected_by_percent'], ascending=False).head(15)

,name,element,value,team,position_id,selected_by_percent,position,predicted_total_points_next_gameweek
26,Gabriel Fernando de Jesus,28,8.0,Arsenal,4,75.2,FWD,60.160
316,Mohamed Salah,283,13.0,Liverpool,3,61.0,MID,79.300
318,Trent Alexander-Arnold,285,7.5,Liverpool,2,59.1,DEF,44.325
342,João Cancelo,306,7.0,Man City,2,45.4,DEF,31.780
353,Erling Håland,318,11.5,Man City,4,44.9,FWD,51.635
160,Reece James,146,6.0,Chelsea,2,39.9,DEF,23.940
418,Neco Williams,295,4.0,Nott'm Forest,2,28.8,DEF,11.520
500,Ivan Perišić,448,5.5,Spurs,2,27.9,DEF,15.345
479,Harry Kane,427,11.5,Spurs,4,27.8,FWD,31.970
245,Andreas Hoelgebaum Pereira,346,4.5,Fulham,3,25.5,MID,11.475


In [387]:
df_start['predicted_total_points_next_gameweek'] = df_start['value'] * df_start['selected_by_percent'] / 10

In [388]:
decisions, captain_decisions, sub_decisions = select_team(df_start.predicted_total_points_next_gameweek.values, df_start.value.values, df_start.position_id.values, df_start.team.values, total_budget=100, sub_factor=0.3)

d:\studies\informatyka\semestr_3\fpl_ai_manager\venv\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Total expected score = 450.15799999999996


In [389]:
print_selected_team(df_start, decisions, captain_decisions, sub_decisions)

**Gabriel Martinelli Silva** Predicted points = 12.36, Price = 6.0, Position = MID

**Gabriel Fernando de Jesus** Predicted points = 60.16, Price = 8.0, Position = FWD

**Reece James** Predicted points = 23.94, Price = 6.0, Position = DEF

**Edouard Mendy** Predicted points = 10.55, Price = 5.0, Position = GK

**Mohamed Salah** Predicted points = 79.3, Price = 13.0, Position = MID

...

**Danny Ward** Predicted points = 7.88, Price = 4.0, Position = GK

**Sam Greenwood** Predicted points = 11.03, Price = 4.5, Position = FWD

**Pedro Lomba Neto** Predicted points = 10.23, Price = 5.5, Position = MID


Captain:


**Mohamed Salah** Predicted points = 79.3, Price = 13.0, Position = MID

## Get transfer for second gameweek

In [15]:
model_path = '/models/baseline/model_mlp_rolling_opponent_stats_features.h5'
sclaer_path = '/models/baseline/SCALER-model_mlp_rolling_opponent_stats_features.gz'

In [17]:
from src.squad_selection.predict_and_select import predict_and_select_transfer

predict_and_select_transfer('2022-23', 1, model_path, sclaer_path, download_newest_data=False)

Are there any NaN values? - False


d:\studies\informatyka\semestr_3\fpl_ai_manager\venv\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Solver status: 1

Selected Transfers:


Transferred out: **Mohamed Salah**, Predicted points = 14.72, Price = 13.0

Transferred in: **Son Heung-min**, Predicted points = 45.24, Price = 12.0

...

**Edouard Mendy** Predicted points = 9.94, Price = 5.0, Position = GK

**Sam Greenwood** Predicted points = 4.15, Price = 4.5, Position = FWD

**Neco Williams** Predicted points = 6.97, Price = 4.0, Position = DEF


Captain:


**Son Heung-min** Predicted points = 45.24, Price = 12.0, Position = MID

#### FOR PREDICTING TRANSFERS ON GW2, OLD MODEL WAS USED!!!

## Get transfer for third gameweek

In [8]:
model_path = ['/models/model_mlp_gk.h5', '/models/model_mlp_field.h5']
scaler_path = ['/models/scaler_gk.gz', '/models/scaler_field.gz']

In [9]:
from src.squad_selection.predict_and_select import predict_and_select_transfer

predict_and_select_transfer('2022-23', 2, model_path, scaler_path, download_newest_data=False, get_actual_chance_playing=True, position_separated=True, budget_now=1)

Are there any NaN values? - False
Are there any NaN values? - False


d:\studies\informatyka\semestr_3\fpl_ai_manager\venv\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Solver status: 1

Selected Transfers:


Transferred in: **Mohamed Salah**, Predicted points = 21.34, Price = 13.0

Transferred out: **Son Heung-min**, Predicted points = 18.31, Price = 12.0

...

**Danny Ward** Predicted points = 9.98, Price = 4.0, Position = GK

**Andreas Hoelgebaum Pereira** Predicted points = 6.66, Price = 4.5, Position = MID

**Ivan Perišić** Predicted points = 7.81, Price = 5.5, Position = DEF


Captain:


**Mohamed Salah** Predicted points = 21.34, Price = 13.0, Position = MID

## Get transfer for GW 4

In [5]:
model_path = ['/models/model_mlp_gk.h5', '/models/model_mlp_field.h5']
scaler_path = ['/models/scaler_gk.gz', '/models/scaler_field.gz']

In [6]:
from src.squad_selection.predict_and_select import predict_and_select_transfer

top_15_predictions = predict_and_select_transfer('2022-23', 3, model_path, scaler_path, download_newest_data=False, get_actual_chance_playing=True, position_separated=True, budget_now=0)

Are there any NaN values? - False
Are there any NaN values? - False


d:\studies\informatyka\semestr_3\fpl_ai_manager\venv\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Solver status: 1

Selected Transfers:


Transferred in: **Oleksandr Zinchenko**, Predicted points = 12.01, Price = 5.2

Transferred out: **Ivan Perišić**, Predicted points = 10.31, Price = 5.4

...

**Danny Ward** Predicted points = 10.03, Price = 4.0, Position = GK

**Neco Williams** Predicted points = 9.67, Price = 4.0, Position = DEF

**Andreas Hoelgebaum Pereira** Predicted points = 9.35, Price = 4.5, Position = MID


Captain:


**Mohamed Salah** Predicted points = 21.62, Price = 13.0, Position = MID

In [7]:
top_15_predictions

,name,element,position,value,team,GW,season,predicted_total_points_next_gameweek,next_3_GWs_matches,opponent_next_gameweek,position_id
282,Mohamed Salah,283,MID,13.0,Liverpool,3,2022-23,21.621765,3,"[Bournemouth, Newcastle United, Everton]",3
317,Erling Haaland,318,FWD,11.7,Manchester City,3,2022-23,20.140432,3,"[Crystal Palace, Nottingham Forest, Aston Villa]",4
300,Kevin De Bruyne,301,MID,12.1,Manchester City,3,2022-23,19.818817,3,"[Crystal Palace, Nottingham Forest, Aston Villa]",3
426,Harry Kane,427,FWD,11.4,Tottenham,3,2022-23,17.923523,3,"[Nottingham Forest, West Ham, Fulham]",4
27,Gabriel Fernando de Jesus,28,FWD,8.1,Arsenal,3,2022-23,17.838783,3,"[Fulham, Aston Villa, Manchester United]",4
427,Son Heung-min,428,MID,11.9,Tottenham,3,2022-23,17.557598,3,"[Nottingham Forest, West Ham, Fulham]",3
18,Gabriel Martinelli Silva,19,MID,6.3,Arsenal,3,2022-23,15.384300,3,"[Fulham, Aston Villa, Manchester United]",3
284,Trent Alexander-Arnold,285,DEF,7.5,Liverpool,3,2022-23,15.372221,3,"[Bournemouth, Newcastle United, Everton]",2
303,Raheem Sterling,304,MID,10.0,Chelsea,3,2022-23,14.837513,3,"[Leicester, Southampton, West Ham]",3
445,Dejan Kulusevski,446,MID,8.2,Tottenham,3,2022-23,14.678350,3,"[Nottingham Forest, West Ham, Fulham]",3


## Get transfer for GW 5

In [16]:
model_path = ['/models/model_mlp_gk.h5', '/models/model_mlp_field.h5']
scaler_path = ['/models/scaler_gk.gz', '/models/scaler_field.gz']

In [17]:
from src.squad_selection.predict_and_select import predict_and_select_transfer

top_15_predictions = predict_and_select_transfer('2022-23', 4, model_path, scaler_path, download_newest_data=False, get_actual_chance_playing=True, position_separated=True, budget_now=0.3)

Are there any NaN values? - False
Are there any NaN values? - False


d:\studies\informatyka\semestr_3\fpl_ai_manager\venv\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Solver status: 1

Selected Transfers:


Transferred out: **Oleksandr Zinchenko**, Predicted points = 6.84, Price = 5.3

Transferred in: **Kieran Trippier**, Predicted points = 11.36, Price = 5.1

...

**Danny Ward** Predicted points = 10.12, Price = 4.0, Position = GK

**Neco Williams** Predicted points = 9.22, Price = 4.1, Position = DEF

**Pedro Lomba Neto** Predicted points = 9.31, Price = 5.3, Position = MID


Captain:


**Mohamed Salah** Predicted points = 21.57, Price = 13.0, Position = MID

In [18]:
top_15_predictions

,name,element,position,value,team,GW,season,predicted_total_points_next_gameweek,next_3_GWs_matches,opponent_next_gameweek,position_id
282,Mohamed Salah,283,MID,13.0,Liverpool,4,2022-23,21.568079,3,"[Newcastle United, Everton, Wolverhampton Wand...",3
317,Erling Haaland,318,FWD,11.7,Manchester City,4,2022-23,19.529243,3,"[Nottingham Forest, Aston Villa, Tottenham]",4
300,Kevin De Bruyne,301,MID,12.2,Manchester City,4,2022-23,19.342487,3,"[Nottingham Forest, Aston Villa, Tottenham]",3
27,Gabriel Fernando de Jesus,28,FWD,8.2,Arsenal,4,2022-23,17.909311,3,"[Aston Villa, Manchester United, Everton]",4
18,Gabriel Martinelli Silva,19,MID,6.4,Arsenal,4,2022-23,16.568823,3,"[Aston Villa, Manchester United, Everton]",3
224,Rodrigo Moreno,225,MID,6.4,Leeds,4,2022-23,16.398438,3,"[Everton, Brentford, Nottingham Forest]",3
426,Harry Kane,427,FWD,11.4,Tottenham,4,2022-23,16.309010,3,"[West Ham, Fulham, Manchester City]",4
427,Son Heung-min,428,MID,11.9,Tottenham,4,2022-23,15.855158,3,"[West Ham, Fulham, Manchester City]",3
303,Raheem Sterling,304,MID,9.9,Chelsea,4,2022-23,14.990455,3,"[Southampton, West Ham, Fulham]",3
292,Luis Díaz,293,MID,8.0,Liverpool,4,2022-23,14.693094,3,"[Newcastle United, Everton, Wolverhampton Wand...",3


## Get transfer for GW 6

In [5]:
model_path = ['/models/model_mlp_gk.h5', '/models/model_mlp_field.h5']
scaler_path = ['/models/scaler_gk.gz', '/models/scaler_field.gz']

In [7]:
from src.squad_selection.predict_and_select import predict_and_select_transfer

top_15_predictions = predict_and_select_transfer('2022-23', 5, model_path, scaler_path, download_newest_data=True, get_actual_chance_playing=True, position_separated=True, budget_now=0.4)

Download complete
start scraping missing teams data season:  2022-23
End of scraping
Are there any NaN values? - False
Are there any NaN values? - False


d:\studies\informatyka\semestr_3\fpl_ai_manager\venv\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Solver status: 1

Selected Transfers:


Transferred out: **Reece James**, Predicted points = 5.99, Price = 6.1

Transferred in: **Ivan Perišić**, Predicted points = 10.9, Price = 5.6

...

**Sam Greenwood** Predicted points = 1.3, Price = 4.5, Position = FWD

**Andreas Hoelgebaum Pereira** Predicted points = 10.09, Price = 4.5, Position = MID

**Pedro Lomba Neto** Predicted points = 8.58, Price = 5.3, Position = MID


Captain:


**Mohamed Salah** Predicted points = 20.69, Price = 13.0, Position = MID

In [ ]:
top_15_predictions

## Get transfer for GW 7

In [5]:
model_path = ['/models/model_mlp_gk.h5', '/models/model_mlp_field.h5']
scaler_path = ['/models/scaler_gk.gz', '/models/scaler_field.gz']

In [6]:
from src.squad_selection.predict_and_select import predict_and_select_transfer

top_15_predictions = predict_and_select_transfer('2022-23', 6, model_path, scaler_path, download_newest_data=True, get_actual_chance_playing=True, position_separated=True, budget_now=0.8)

Download complete
start scraping missing teams data season:  2022-23
200
End of scraping
Are there any NaN values? - False
Are there any NaN values? - False


d:\studies\informatyka\semestr_3\fpl_ai_manager\venv\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Solver status: 1

Selected Transfers:


Transferred in: **Pascal Groß**, Predicted points = 14.03, Price = 5.9

Transferred out: **Pedro Lomba Neto**, Predicted points = 8.41, Price = 5.2

...

**Danny Ward** Predicted points = 10.06, Price = 4.0, Position = GK

**Andreas Hoelgebaum Pereira** Predicted points = 9.74, Price = 4.6, Position = MID

**Ivan Perišić** Predicted points = 6.07, Price = 5.6, Position = DEF


Captain:


**Mohamed Salah** Predicted points = 20.64, Price = 13.0, Position = MID

In [ ]:
top_15_predictions